In [12]:
#Import dependancies
import warnings
import numpy as np
import pandas as pd

In [13]:
#Import datafiles
links = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\links.csv')
movies = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\movies.csv')
ratings = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\ratings.csv')
tags = pd.read_csv(r'C:\Users\shubham\Downloads\ml-latest-small\ml-latest-small\tags.csv')

In [14]:
movies.drop('genres', axis=1, inplace=True)
ratings.drop('timestamp', axis=1, inplace=True)

In [15]:
#merging data to filter useful info
Movie_data = pd.merge(ratings, movies, on='movieId')

In [16]:
Movie_data.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [17]:
#Normalizing ratings for users
Grouped_users = Movie_data.groupby(by='userId')
user_means = Grouped_users['rating'].agg(['mean', 'count'])

In [18]:
#Calculating mean centered rating
mean_centering = lambda ratings: ratings - ratings.mean()
Movie_data['Mean_Rating'] = Grouped_users['rating'].transform(mean_centering)
Movie_data.head()

,userId,movieId,rating,title,Mean_Rating
0,1,1,4.0,Toy Story (1995),-0.366379
1,5,1,4.0,Toy Story (1995),0.363636
2,7,1,4.5,Toy Story (1995),1.269737
3,15,1,2.5,Toy Story (1995),-0.948148
4,17,1,4.5,Toy Story (1995),0.290476


In [19]:
Total_users = len(sorted(set(Movie_data['userId'])))
Total_movies = len(sorted(set(Movie_data['movieId'])))

In [20]:
#Function to calculate similarity score and recommend movies
def recommend_movies(user_rating,Movie_matrix):
    Matrix_correlation = Movie_matrix.corr(method='pearson', min_periods=80)
    similar_candidates = pd.Series()
    for i in range(0, len(user_rating.index)):
        similar_movies = Matrix_correlation[user_rating.index[i]].dropna()
        similar_movies = similar_movies.map(lambda x: x * user_rating[i])
        similar_candidates = similar_candidates.append(similar_movies)
    similar_candidates.sort_values(inplace = True, ascending = False)
    similar_candidates = similar_candidates.groupby(similar_candidates.index).sum()
    similar_candidates.sort_values(inplace = True, ascending = False)
    return similar_candidates.head(10)


In [21]:
#Normalization of movie user matrix
movie_matrix_normalized = Movie_data.pivot_table(index='userId', columns='title', values='Mean_Rating')
myRatings_normalized = movie_matrix_normalized.loc[1].dropna()

In [22]:
recommend(myRatings_normalized,movie_matrix_normalized)

Star Wars: Episode IV - A New Hope (1977)                                         4.028704
Star Wars: Episode V - The Empire Strikes Back (1980)                             3.715284
Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)    3.091970
Star Wars: Episode VI - Return of the Jedi (1983)                                 3.054864
American Beauty (1999)                                                            2.834594
Godfather, The (1972)                                                             2.603807
Fargo (1996)                                                                      2.363469
Back to the Future (1985)                                                         2.269074
Reservoir Dogs (1992)                                                             2.246389
Seven (a.k.a. Se7en) (1995)                                                       2.241820
dtype: float64